# Linear Regression on the Boston Housing dataset 

## Import required libraries

In [1]:
import numpy as np
import logging
import sys
from enum import Enum
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold

logging.basicConfig(stream=sys.stderr, level=logging.INFO)
plt.style.use("seaborn-darkgrid")
%matplotlib inline
%config InlineBackend.figure_format='retina'

### Load the Boston Housing dataset

In [2]:
X, y = load_boston(return_X_y=True)

## Information on our dataset

Boston house prices dataset has 506 instances and for each instance, it has 13 attributes
and one target value.

-   CRIM per capita crime rate by town
-   ZN proportion of residential land zoned for lots over 25,000 sq.ft.
-   INDUS proportion of non-retail business acres per town
-   CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
-   NOX nitric oxides concentration (parts per 10 million)
-   RM average number of rooms per dwelling
-   AGE proportion of owner-occupied units built prior to 1940
-   DIS weighted distances to five Boston employment centres
-   RAD index of accessibility to radial highways
-   TAX full-value property-tax rate per \$10,000
-   PTRATIO pupil-teacher ratio by town B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
-   LSTAT % lower status of the population
-   MEDV Median value of owner-occupied homes in $1000’s 

In [3]:
print(f"X input shape is {X.shape}, y target shape is {y.shape}")

X input shape is (506, 13), y target shape is (506,)


## Linear Regression Class and Utility classes

In [4]:
class TrainType(Enum):
    NORMAL_EQUATION = 0
    GRADIENT_DESCENT = 1


Model_Parameter = namedtuple(
    "Model_Parameter", ("lambda_ridge, training_method, alpha, epochs")
)

In [5]:
class Preprocessing:
    def __init__(self):
        self.mean = None
        self.std = None

    @staticmethod
    def standardize(X, mean=None, std=None, inplace=False):
        if mean is None:
            mean = np.mean(X, axis=0)
        if std is None:
            std = np.std(X, axis=0)

        std = np.where(std == 0, 1, std)
        if inplace:
            X -= mean
            X /= std
        else:
            X = (X - mean) / std
        return X

    @staticmethod
    def insert_bias_term(X):
        bias_arr = np.ones(X.shape[0])
        return np.c_[bias_arr, X]

    def standardize_save_state(self, X, mean=None, std=None, inplace=False):
        if mean is None:
            mean = np.mean(X, axis=0)
        if std is None:
            std = np.std(X, axis=0)

        std = np.where(std == 0, 1, std)
        self.mean = mean
        self.std = std
        if inplace:
            X -= mean
            X /= std
        else:
            X = (X - mean) / std
        return X

    def fit(self, X, inplace=False):
        if self.mean is None or self.std is None:
            raise ValueError("Mean or std is not for the preprocessing object")
        if inplace:
            X -= self.mean
            X /= self.std
        else:
            X = (X - self.mean) / self.std
        return X

In [6]:
class LinearRegr:
    __slots__ = ["theta"]

    def __init__(self):
        self.theta = None

    def __repr__(self):
        return " ".join([str(parm) for parm in np.ndarray.flatten(self.theta)])

    def fit(
        self,
        X,
        y,
        model_params=Model_Parameter(
            lambda_ridge=0,
            training_method=TrainType.NORMAL_EQUATION,
            alpha=0.5,
            epochs=1000,
        ),
    ):
        """
        Fit/train the linear model
        It has been assumed that the bias has been added to X
        """
        if X.shape[0] != y.shape[0]:
            raise ValueError(
                f"X shape {X.shape[0]} != y shape {y.shape[0]}. Dimensions not matching"
            )

        if model_params.training_method == TrainType.NORMAL_EQUATION:
            self._normal_equation_method(X, y, model_params)
        elif model_params.training_method == TrainType.GRADIENT_DESCENT:
            self._gradient_descent_method(X, y, model_params)
        else:
            raise ValueError("Model type not supplied")

    def _normal_equation_method(self, X, y, model_params):
        # Feature Scaling is not required
        # theta = (XtX + LE*)-1 . Xt.y
        # Almost identity matrix E where the first row, first col elem is 0
        # since we do not regularize the bias input, x0 = 1
        lambda_ridge = model_params.lambda_ridge

        E_start = np.identity(X.shape[1])
        E_start[0][0] = 0
        E_start *= lambda_ridge
        X_t = np.matrix.transpose(X)

        dot_Xt_X = np.dot(X_t, X)  # XtX
        self.theta = np.dot(np.dot(np.linalg.pinv(dot_Xt_X + E_start), X_t), y)

    def _gradient_descent_method(self, X, y, model_params):
        """
        WARNING Feature scaling should already be done for X
        Batch Gradient Descent
        """
        lambda_ridge, training_method, alpha, epochs = model_params
        self.theta = np.zeros(X.shape[1])
        loss_overtime = []
        m = y.shape[0]

        for epoch in range(epochs):
            gradient_wout_regu = (1 / m) * np.dot(
                np.matrix.transpose(X), np.dot(X, self.theta) - y
            )
            # 0th parameter/bias is not regularized
            self.theta[0] = self.theta[0] - alpha * gradient_wout_regu[0]
            gradient_with_regu = gradient_wout_regu + ((lambda_ridge / m) * self.theta)
            # All other parameters regularized
            self.theta[1:] = self.theta[1:] - alpha * gradient_with_regu[1:]

            if epoch % 1 == 0:
                current_loss = self.loss(X, y, lambda_ridge)
                logging.info(f"Current loss at epoch {epoch} is {current_loss}")
                loss_overtime.append(current_loss)

        self.plot_loss_curve(loss_overtime, epochs)

    def plot_loss_curve(self, loss_arr, iterations, log_mode=False):
        if log_mode:
            plt.semilogx(range(iterations), loss_arr)
        else:
            plt.plot(loss_arr)
        plt.title("Loss function")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.grid(True)
        plt.show()

    @staticmethod
    def mse_loss(X, y, theta, lambda_ridge=0):
        """Calculates the MSE loss for linear regression"""
        if X.shape[0] != y.shape[0]:
            raise ValueError(
                f"X shape {X.shape[0]} != y shape {y.shape[0]}. Dimensions not matching"
            )
        elif X.shape[1] != theta.shape[0]:
            raise ValueError(
                f"X shape {X.shape[1]} != theta shape {theta.shape[0]}. Dimensions not matching"
            )

        X_theta_minus_y = np.dot(X, theta) - y
        X_theta_minus_y_t = np.matrix.transpose(X_theta_minus_y)

        return (1 / (2 * X.shape[0])) * (
            (np.dot(X_theta_minus_y_t, X_theta_minus_y))
            + (lambda_ridge * np.dot(np.matrix.transpose(theta[1:]), theta[1:]))
        )

    @staticmethod
    def predict_X(X, theta):
        """
        Predict using the linear model
        """
        if theta is None:
            raise ValueError("Model has not been trained yet")

        # prediction = X*theta
        return np.dot(X, theta)

    @staticmethod
    def score_X(X, y, theta):
        """
        Returns the coefficient of determination
        """
        if theta is None:
            raise ValueError("Model has not been trained yet")

        y_mean = np.mean(y)
        y_pred = LinearRegr.predict_X(X, theta)
        ss_total = sum((y - y_mean) ** 2)  # total sum of squares
        ss_res = sum((y - y_pred) ** 2)  # sum of squared residuals

        return 1 - (ss_res / ss_total)

    def loss(self, X, y, lambda_ridge=0):
        """
        Calculates the current loss
        """
        if self.theta is None:
            raise ValueError("Model has not been trained yet")

        return LinearRegr.mse_loss(X, y, self.theta, lambda_ridge)

    def score(self, X, y):
        """
        Returns the coefficient of determination
        """
        return LinearRegr.score_X(X, y, self.theta)

    def predict(self, X):
        """
        Predict using the linear model
        """
        return LinearRegr.predict_X(X, self.theta)

In [7]:
class KFoldCrossValidator:
    __slots__ = ["train_loss", "test_loss", "theta"]

    def __init__(self):
        self.train_loss = []
        self.test_loss = []
        self.theta = None

    def cross_validate(
        self,
        model,
        model_params,
        X,
        y,
        k=10,
        custom_kfold=False,
        seed=np.random.randint(10000),
    ):
        """
        Cross validation function, the theta parameter chosen is from the split with the least test error
        """

        m = X.shape[0]
        lambda_ridge, training_method, alpha, epochs = model_params
        min_test_error = float("inf")  # tracks the minimum error with k-folds
        best_fit_theta = None  # saves the best theta value with the min_test_error

        if custom_kfold:
            logging.info(
                f"Running Custom KFoldCrossValidator with {k} folds and lambda={lambda_ridge}"
            )
            np.random.seed(seed)  # seed random shuffler
            if m < k:
                raise ValueError(
                    f"No of k splits {k} cannot be greater than no. of samples {m}"
                )

            # Randomly shuffle X and y inplace while matching corresponding feat and target
            for i in range(m):
                swap_idx = np.random.randint(i, m)
                # ensures the corresponding feat-target values match
                X[[i, swap_idx]] = X[[swap_idx, i]]
                y[[i, swap_idx]] = y[[swap_idx, i]]

            # test start and end idx
            fold_step = m // k
            start = 0
            end = fold_step
            for i in range(k):
                end = min(end, m)  # prevent array idx out of bounds
                X_train, X_test = (
                    np.concatenate([X[0:start], X[end:m]], axis=0),
                    X[start:end],
                )
                y_train, y_test = (
                    np.concatenate([y[0:start], y[end:m]], axis=0),
                    y[start:end],
                )
                start += fold_step
                end += fold_step

                model.fit(X_train, y_train, model_params)
                cur_train_loss = model.loss(X_train, y_train, lambda_ridge)
                cur_test_loss = model.loss(X_test, y_test, lambda_ridge)
                self.train_loss.append(cur_train_loss)
                self.test_loss.append(cur_test_loss)

                if cur_test_loss < min_test_error:
                    min_test_error = cur_test_loss
                    best_fit_theta = model.theta
        else:
            logging.info(
                f"Running Sklearn KFoldCrossValidator with {k} folds and lambda {lambda_ridge}"
            )
            kf = KFold(n_splits=k, random_state=seed, shuffle=True)
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                model.fit(X_train, y_train, model_params)
                cur_train_loss = model.loss(X_train, y_train, lambda_ridge)
                cur_test_loss = model.loss(X_test, y_test, lambda_ridge)
                self.train_loss.append(cur_train_loss)
                self.test_loss.append(cur_test_loss)

                if cur_test_loss < min_test_error:
                    min_test_error = cur_test_loss
                    best_fit_theta = model.theta
        self.theta = best_fit_theta

## Running our regression model

**Steps taken**

-   Create a linear regression model class
-   Create a KFoldCrossValidator class
-   Add the bias feature
-   Create a model parameter object with the hyperparameter values and the model type
-   Run the cross validation function to get average loss on the train and test set

## Linear regression model with no regularization i.e. lambda=0

Now we fit a linear regression model using the closed form solution. Then use k-fold cross validation to estimate the performance of this model. Print the average of your recorded scores for both the test set and training set.

**Average training and test loss with 10 folds Cross Validation and a lambda of 0 with the normal equation (Closed form solution)**

In [8]:
bh_model = LinearRegr()
kfold_linear_regr = KFoldCrossValidator()
X_feat = Preprocessing.insert_bias_term(X)

model_params = Model_Parameter(
    lambda_ridge=0, training_method=TrainType.NORMAL_EQUATION, alpha=0, epochs=0
)
kfold_linear_regr.cross_validate(
    bh_model, model_params, X_feat, y, k=10, custom_kfold=False
)

lregr_train_loss = kfold_linear_regr.train_loss
lregr_test_loss = kfold_linear_regr.test_loss
print(f"Average train loss is: {sum(lregr_train_loss) / len(lregr_train_loss)}")
print(f"Average test loss is: {sum(lregr_test_loss) / len(lregr_test_loss)}")

print(f"R squared for the entire dataset is {bh_model.score(X_feat, y)}")

INFO:root:Running Sklearn KFoldCrossValidator with 10 folds and lambda 0


Average train loss is: 10.905187135839643
Average test loss is: 11.787812931304178
R squared for the entire dataset is 0.7402547552453309


## Ridge regression model with regularization. lambda parameters tested using K-Fold cross validation

In [9]:
logger = logging.getLogger()
logger.disabled = True

possible_lambda_ridge = np.logspace(1, 7, num=13)
min_test_error = float("inf")  # tracks the minimum error with k-folds
best_lambda = None  # saves the best lambda value with the min_test_error
best_theta = None  # saves the best theta value with the min_test_error

for lambda_ridge in possible_lambda_ridge:
    bh_model = LinearRegr()
    kfold_linear_regr = KFoldCrossValidator()
    X_feat = Preprocessing.insert_bias_term(X)

    model_params = Model_Parameter(
        lambda_ridge=lambda_ridge,
        training_method=TrainType.NORMAL_EQUATION,
        alpha=0,
        epochs=0,
    )
    kfold_linear_regr.cross_validate(
        bh_model, model_params, X_feat, y, k=10, custom_kfold=False
    )

    lregr_train_loss = kfold_linear_regr.train_loss
    lregr_test_loss = kfold_linear_regr.test_loss
    print(f"When lambda is {lambda_ridge}")
    avg_train_loss, avg_test_loss = (
        sum(lregr_train_loss) / len(lregr_train_loss),
        sum(lregr_test_loss) / len(lregr_test_loss),
    )
    print(f"Average train loss is: {avg_train_loss}")
    print(f"Average test loss is: {avg_test_loss}")
    print()

    if avg_test_loss < min_test_error:
        min_test_error = avg_test_loss
        best_lambda = lambda_ridge
        best_theta = bh_model.theta

print(f"The lambda for ridge regression that yields the mininum error is {best_lambda}")
print(
    f"R squared for the entire dataset is {LinearRegr.score_X(X_feat, y, best_theta)}"
)
logger.disabled = False

When lambda is 10.0
Average train loss is: 11.573173093895647
Average test loss is: 14.547718368863931

When lambda is 31.622776601683793
Average train loss is: 12.008203365913863
Average test loss is: 16.8980895294772

When lambda is 100.0
Average train loss is: 12.805385012208115
Average test loss is: 20.517681871381875

When lambda is 316.22776601683796
Average train loss is: 14.032687959712714
Average test loss is: 24.842894898856844

When lambda is 1000.0
Average train loss is: 15.820055352459372
Average test loss is: 31.506953974551795

When lambda is 3162.2776601683795
Average train loss is: 18.497448858769012
Average test loss is: 41.55202619784027

When lambda is 10000.0
Average train loss is: 22.234452430655153
Average test loss is: 51.25777463326127

When lambda is 31622.776601683792
Average train loss is: 26.117030701561685
Average test loss is: 50.47470198003427

When lambda is 100000.0
Average train loss is: 28.837691733164384
Average test loss is: 44.01005380124742

When

A lambda of 10 yields the best results quantified by the minimum test error, that is: 

-   Average train loss is: 11.56861911239611
-   Average test loss is: 14.642007589269346

## Linear regression extended to Polynomial regression

In [10]:
two_degree_transform = PolynomialFeatures(2)
X_feat_poly = two_degree_transform.fit_transform(X)

Average training and test loss with 10 folds Cross Validation and a lambda of 0 with the normal equation (Closed form solution)

In [12]:
bh_poly_model = LinearRegr()
kfold_poly_regr = KFoldCrossValidator()

poly_model_params = Model_Parameter(
    lambda_ridge=0, training_method=TrainType.NORMAL_EQUATION, alpha=0, epochs=0
)
kfold_poly_regr.cross_validate(
    bh_poly_model, poly_model_params, X_feat_poly, y, k=10, custom_kfold=False
)

pregr_train_loss = kfold_poly_regr.train_loss
pregr_test_loss = kfold_poly_regr.test_loss
print(f"Average train loss is: {sum(pregr_train_loss) / len(pregr_train_loss)}")
print(f"Average test loss is: {sum(pregr_test_loss) / len(pregr_test_loss)}")
print(f"R squared for the entire dataset is {bh_poly_model.score(X_feat_poly, y)}")

INFO:root:Running Sklearn KFoldCrossValidator with 10 folds and lambda 0


Average train loss is: 2.9982571275911836
Average test loss is: 7.583571795125268
R squared for the entire dataset is 0.9214961827244105


## Polynomial regression model with regularization. lambda parameters tested using K-Fold cross validation

In [13]:
logger = logging.getLogger()
logger.disabled = True

possible_lambda_ridge = np.logspace(1, 7, num=13)
min_test_error = float("inf")  # tracks the minimum error with k-folds
best_poly_lambda = None  # saves the lambda theta value with the min_test_error
best_theta = None  # saves the best theta value with the min_test_error

for lambda_ridge in possible_lambda_ridge:
    bh_poly_model = LinearRegr()
    kfold_poly_regr = KFoldCrossValidator()

    poly_model_params = Model_Parameter(
        lambda_ridge=lambda_ridge,
        training_method=TrainType.NORMAL_EQUATION,
        alpha=0,
        epochs=0,
    )
    kfold_poly_regr.cross_validate(
        bh_poly_model, poly_model_params, X_feat_poly, y, k=10, custom_kfold=False
    )

    pregr_train_loss = kfold_poly_regr.train_loss
    pregr_test_loss = kfold_poly_regr.test_loss

    print(f"When lambda is {lambda_ridge}")
    avg_train_loss, avg_test_loss = (
        sum(pregr_train_loss) / len(pregr_train_loss),
        sum(pregr_test_loss) / len(pregr_test_loss),
    )
    print(f"Average train loss is: {avg_train_loss}")
    print(f"Average test loss is: {avg_test_loss}")
    print()

    if avg_test_loss < min_test_error:
        min_test_error = avg_test_loss
        best_poly_lambda = lambda_ridge
        best_theta = bh_poly_model.theta

print(
    f"The lambda for poylnomial regression that yields the mininum error is {best_poly_lambda}"
)
print(
    f"R squared for the entire dataset is {LinearRegr.score_X(X_feat_poly, y, best_theta)}"
)
logger.disabled = False

When lambda is 10.0
Average train loss is: 3.5688836719411916
Average test loss is: 8.861704861858973

When lambda is 31.622776601683793
Average train loss is: 3.794702043557172
Average test loss is: 9.060617269450457

When lambda is 100.0
Average train loss is: 4.026295922630787
Average test loss is: 9.1569351860967

When lambda is 316.22776601683796
Average train loss is: 4.27034329377639
Average test loss is: 9.40068844560397

When lambda is 1000.0
Average train loss is: 4.544846042803544
Average test loss is: 9.765489137321014

When lambda is 3162.2776601683795
Average train loss is: 4.832409382189917
Average test loss is: 10.112162306092737

When lambda is 10000.0
Average train loss is: 5.125969050514972
Average test loss is: 10.570889887669308

When lambda is 31622.776601683792
Average train loss is: 5.473409845191977
Average test loss is: 11.234166599495888

When lambda is 100000.0
Average train loss is: 5.915900027002763
Average test loss is: 12.060589692747646

When lambda is 

## Choice of the better model

From our results if we want to decide on our test set error results alone, the polynomial regression model with two polynomial features (i.e. form \[a,b\] generates \[1, a, b, a^2, ab, b^2\]) and a lambda of 10.0 generates the lowest test error and the highest R squared value of 0.91. However, this might just be a case of our model overfitting on our dataset due the high number of features and the very limited size of the training set (Only 506 values).